# DOG BREED CLASSIFICATION USING TRANSFER LEARNING

In [1]:
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
# download dataset
!kaggle datasets download -d 'mohamedchahed/dog-breeds'
# unzip dataset
!unzip dog-breeds.zip

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# Define the initial path
initial_path = '/content/dog-breeds'

# Define the new top-level directory names
directories = ['train', 'test']

# Define the subdirectory names for each top-level directory
subdirectories = ['beagle', 'bulldog', 'dalmatian', 'german-shepherd', 'husky', 'labrador-retriever', 'poodle', 'rottweiler']

# Define the proportions for train, validation, and test sets
props = [0.7, 0.3]

# Loop over the top-level directories and subdirectories, and create the corresponding directories
for directory in directories:
    for subdirectory in subdirectories:
        path = os.path.join(initial_path, subdirectory)
        files = os.listdir(path)
        random.shuffle(files)
        total = len(files)
        train_end = int(total * props[0])
        val_end = train_end + int(total * props[1])
        if directory == 'train':
            new_files = files[:train_end]
        elif directory == 'val':
            new_files = files[train_end:val_end]
        else:
            new_files = files[val_end:]
        new_path = os.path.join(directory, subdirectory)
        os.makedirs(new_path, exist_ok=True)
        for file in new_files:
            old_file_path = os.path.join(path, file)
            new_file_path = os.path.join(new_path, file)
            shutil.copy(old_file_path, new_file_path)

In [5]:
train_dir = 'train'
test_dir = 'test'

# Define the image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 32
# Define the data generators for the train, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    color_mode='rgb')


test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                color_mode='rgb')

Found 376 images belonging to 8 classes.
Found 7 images belonging to 8 classes.


#VGG 16

In [6]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense , Flatten
from tensorflow.keras.models import Model

In [7]:
vgg = VGG16(include_top = False,input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
for layer in vgg.layers:
  print(layer)

In [10]:
for layer in vgg.layers:
  layer.trainable = False

In [11]:
x = Flatten()(vgg.output)

In [12]:
output = Dense(8,activation = 'softmax')(x)

In [13]:
vgg16 = Model(vgg.input,output)

In [14]:
vgg16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
vgg16.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [16]:
vgg16.fit(train_generator,validation_data = test_generator,epochs=10 )

Epoch 1/10
12/12 [==============================] - 27s 1s/step - loss: 2.2112 - accuracy: 0.2580 - val_loss: 1.4655 - val_accuracy: 0.4286
Epoch 2/10
12/12 [==============================] - 9s 747ms/step - loss: 1.2505 - accuracy: 0.5798 - val_loss: 0.9771 - val_accuracy: 0.7143
Epoch 3/10
12/12 [==============================] - 8s 679ms/step - loss: 0.8296 - accuracy: 0.7181 - val_loss: 0.9401 - val_accuracy: 0.7143
Epoch 4/10
12/12 [==============================] - 10s 838ms/step - loss: 0.6643 - accuracy: 0.7686 - val_loss: 0.6109 - val_accuracy: 0.8571
Epoch 5/10
12/12 [==============================] - 10s 845ms/step - loss: 0.5785 - accuracy: 0.8191 - val_loss: 0.5487 - val_accuracy: 0.8571
Epoch 6/10
12/12 [==============================] - 9s 710ms/step - loss: 0.4333 - accuracy: 0.8777 - val_loss: 0.2663 - val_accuracy: 0.8571
Epoch 7/10
12/12 [==============================] - 10s 803ms/step - loss: 0.4174 - accuracy: 0.9043 - val_loss: 0.2756 - val_accuracy: 1.0000
Epoch

In [17]:
vgg16.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# RESNET 50

In [18]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense , Flatten
from tensorflow.keras.models import Model

In [19]:
resnet= ResNet50(include_top = False,input_shape=(224,224,3))

94765736/94765736 [==============================] - 1s 0us/step


In [20]:
for layer in resnet.layers:
  print(layer)

In [21]:

for layer in resnet.layers:
  layer.trainable = False

In [22]:
x = Flatten()(resnet.output)

In [23]:
output = Dense(8,activation = 'softmax')(x)

In [24]:
resnet= Model(resnet.input,output)

In [25]:
resnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [26]:
resnet.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [27]:
resnet.fit(train_generator,validation_data = test_generator,epochs=10 )

Epoch 1/10
12/12 [==============================] - 17s 941ms/step - loss: 8.6675 - accuracy: 0.1436 - val_loss: 6.0575 - val_accuracy: 0.1429
Epoch 2/10
12/12 [==============================] - 8s 647ms/step - loss: 6.9999 - accuracy: 0.1356 - val_loss: 3.4116 - val_accuracy: 0.4286
Epoch 3/10
12/12 [==============================] - 8s 636ms/step - loss: 4.2426 - accuracy: 0.1862 - val_loss: 2.2663 - val_accuracy: 0.1429
Epoch 4/10
12/12 [==============================] - 9s 778ms/step - loss: 2.8612 - accuracy: 0.2207 - val_loss: 2.5258 - val_accuracy: 0.2857
Epoch 5/10
12/12 [==============================] - 9s 750ms/step - loss: 2.5177 - accuracy: 0.2340 - val_loss: 1.5863 - val_accuracy: 0.2857
Epoch 6/10
12/12 [==============================] - 8s 652ms/step - loss: 2.2711 - accuracy: 0.2048 - val_loss: 1.5330 - val_accuracy: 0.5714
Epoch 7/10
12/12 [==============================] - 9s 774ms/step - loss: 2.2721 - accuracy: 0.2580 - val_loss: 1.4668 - val_accuracy: 0.4286
Epoch

In [28]:
resnet.save('renet.h5')

# INCEPTION

In [29]:
img_height = 299
img_width = 299
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    color_mode='rgb')


test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                color_mode='rgb')

Found 376 images belonging to 8 classes.
Found 7 images belonging to 8 classes.


In [30]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense , Flatten
from tensorflow.keras.models import Model

In [31]:
inception= InceptionV3(include_top = False,input_shape=(299,299,3))

87910968/87910968 [==============================] - 1s 0us/step


In [32]:
for layer in resnet.layers:
  print(layer)

In [33]:
for layer in resnet.layers:
  layer.trainable = False

In [34]:
x = Flatten()(inception.output)

In [35]:
output = Dense(8,activation = 'softmax')(x)

In [36]:
inception= Model(inception.input,output)

In [37]:
inception.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0]

In [38]:
inception.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [39]:
inception.fit(train_generator,validation_data = test_generator,epochs=10 )

Epoch 1/10
12/12 [==============================] - 74s 2s/step - loss: 10.1741 - accuracy: 0.2128 - val_loss: 19216.6914 - val_accuracy: 0.1429
Epoch 2/10
12/12 [==============================] - 14s 1s/step - loss: 8.7789 - accuracy: 0.1941 - val_loss: 118563.7734 - val_accuracy: 0.1429
Epoch 3/10
12/12 [==============================] - 14s 1s/step - loss: 3.5277 - accuracy: 0.2181 - val_loss: 748522.1250 - val_accuracy: 0.1429
Epoch 4/10
12/12 [==============================] - 14s 1s/step - loss: 3.1099 - accuracy: 0.1809 - val_loss: 136493.8594 - val_accuracy: 0.1429
Epoch 5/10
12/12 [==============================] - 14s 1s/step - loss: 2.9712 - accuracy: 0.1755 - val_loss: 253621.9062 - val_accuracy: 0.2857
Epoch 6/10
12/12 [==============================] - 14s 1s/step - loss: 2.9880 - accuracy: 0.1622 - val_loss: 17096.1543 - val_accuracy: 0.1429
Epoch 7/10
12/12 [==============================] - 14s 1s/step - loss: 2.8311 - accuracy: 0.1489 - val_loss: 826.5106 - val_accura

In [40]:
inception.save('inception.h5')

# XCEPTION

In [41]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    color_mode='rgb')


test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                color_mode='rgb')

Found 376 images belonging to 8 classes.
Found 7 images belonging to 8 classes.


In [42]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense , Flatten
from tensorflow.keras.models import Model

In [43]:
xception= Xception(include_top = False,input_shape=(299,299,3))

83683744/83683744 [==============================] - 1s 0us/step


In [44]:
for layer in xception.layers:
  print(layer)

In [45]:
for layer in xception.layers:
  layer.trainable = False

In [46]:
x = Flatten()(xception.output)

In [47]:
output = Dense(8,activation = 'softmax')(x)

In [48]:
xception= Model(xception.input,output)

In [49]:
xception.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 149, 149, 32)         864       ['input_4[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 149, 149, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 149, 149, 32)         0         ['block1_conv1_bn[0][0]'

In [50]:
xception.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [51]:
xception.fit(train_generator,validation_data = test_generator,epochs=10 )

Epoch 1/10
12/12 [==============================] - 29s 2s/step - loss: 1.0810 - accuracy: 0.8431 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
12/12 [==============================] - 13s 1s/step - loss: 0.2874 - accuracy: 0.9814 - val_loss: 0.0364 - val_accuracy: 1.0000
Epoch 3/10
12/12 [==============================] - 13s 1s/step - loss: 0.1984 - accuracy: 0.9814 - val_loss: 2.8977 - val_accuracy: 0.8571
Epoch 4/10
12/12 [==============================] - 13s 1s/step - loss: 0.2354 - accuracy: 0.9814 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
12/12 [==============================] - 13s 1s/step - loss: 0.1156 - accuracy: 0.9947 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
12/12 [==============================] - 13s 1s/step - loss: 0.0370 - accuracy: 0.9947 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
12/12 [==============================] - 12s 1s/step - loss: 0.0562 - accuracy: 0.9920 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [52]:
xception.save('xception.h5')